# CDX Tranche Pricing - G-VG Copula Model

Mixed Gaussian-Variance Gamma copula with stochastic correlation.

In [ ]:
import numpy as np
import pandas as pd
import json
from scipy.stats import norm, t
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')

## 1. Load Data

In [ ]:
constituents = pd.read_csv('../data/cdx_constituents.csv').dropna(subset=['Company', '5Y_Spread'])
ois_curve = pd.read_csv('../data/ois_curve.csv').dropna(subset=['Tenor'])

with open('../data/cdx_market_data.json', 'r') as f:
    market_data = json.load(f)

with open('../results/results.json', 'r') as f:
    gaussian_results = json.load(f)

print(f"Loaded {len(constituents)} companies")

## 2. Bootstrap Survival Probabilities

In [ ]:
def bootstrap_survival(spread_bps, recovery_rate):
    hazard = (spread_bps / 10000.0) / (1 - recovery_rate)
    return np.exp(-hazard * 5.0)

default_probs = np.array([1 - bootstrap_survival(row['5Y_Spread'], row['Recovery']) 
                          for _, row in constituents.iterrows()])
weights = constituents['Weight'].values

## 3. G-VG Copula

In [ ]:
class GVGCopula:
    def __init__(self, rho_low, rho_high, p_regime, df, n_points=150):
        self.rho_low = rho_low
        self.rho_high = rho_high
        self.p_regime = p_regime
        self.df = df
        
        self.Y = np.linspace(-4, 4, n_points)
        dy = self.Y[1] - self.Y[0]
        
        w_n = norm.pdf(self.Y) * dy
        w_t = t.pdf(self.Y, df) * dy
        
        self.w_low = 0.7 * (w_n / w_n.sum()) + 0.3 * (w_t / w_t.sum())
        self.w_high = 0.3 * (w_n / w_n.sum()) + 0.7 * (w_t / w_t.sum())
    
    def cond_pd(self, PD, Y, rho):
        if PD <= 0 or PD >= 1:
            return max(0, min(1, PD))
        return norm.cdf((norm.ppf(PD) - np.sqrt(rho) * Y) / np.sqrt(1 - rho))
    
    def loss_dist(self, pds, wts):
        lgd = 0.6
        L_low = np.array([lgd * sum(wts * np.array([self.cond_pd(pd, y, self.rho_low) for pd in pds])) 
                         for y in self.Y])
        L_high = np.array([lgd * sum(wts * np.array([self.cond_pd(pd, y, self.rho_high) for pd in pds])) 
                          for y in self.Y])
        
        L = np.concatenate([L_low, L_high])
        P = np.concatenate([self.w_low * self.p_regime, self.w_high * (1 - self.p_regime)])
        return L, P / P.sum()
    
    def tranche_el(self, L, P, a, d):
        return sum(np.maximum(0, np.minimum(L - a, d - a)) / (d - a) * P)

## 4. Pricing Function

In [ ]:
ois_5y = ois_curve[ois_curve['Tenor'] == '5Y']['Mid_Yield'].values[0]
df = np.exp(-ois_5y * np.arange(0.25, 5.25, 0.25))
rpv01 = sum(df * (1 - sum(weights * default_probs)) * 0.25)

def price_all(params):
    copula = GVGCopula(*params)
    L, P = copula.loss_dist(default_probs, weights)
    
    el_eq = copula.tranche_el(L, P, 0.00, 0.03)
    upfront = (el_eq - 500/10000 * rpv01) * 100
    
    spreads = [
        copula.tranche_el(L, P, 0.03, 0.07) / rpv01 * 10000,
        copula.tranche_el(L, P, 0.07, 0.10) / rpv01 * 10000,
        copula.tranche_el(L, P, 0.10, 0.15) / rpv01 * 10000,
        copula.tranche_el(L, P, 0.15, 1.00) / rpv01 * 10000
    ]
    
    return upfront, spreads

## 5. Fast Grid Search Calibration

In [ ]:
market_prices = market_data['market_tranche_prices']
target = [market_prices['equity_0_3_upfront'], market_prices['mezz_3_7'], 
          market_prices['mezz_7_10'], market_prices['senior_10_15'], market_prices['senior_15_100']]

print("Calibrating via grid search (faster)...")

best_err = 1e10
best_params = None

for rho_l in [0.25, 0.30, 0.35, 0.40]:
    for rho_h in [0.55, 0.60, 0.65, 0.70]:
        if rho_h <= rho_l:
            continue
        for p in [0.5, 0.6, 0.7]:
            for df in [4, 6, 8]:
                try:
                    upf, spr = price_all([rho_l, rho_h, p, df])
                    err = abs(upf - target[0]) * 2
                    for i in range(4):
                        err += abs(spr[i] - target[i+1]) / 50
                    
                    if err < best_err:
                        best_err = err
                        best_params = [rho_l, rho_h, p, df]
                        print(f"  ρ={rho_l:.2f}-{rho_h:.2f}, p={p:.1f}, df={df} → err={err:.2f}")
                except:
                    pass

rho_l, rho_h, p_r, df_opt = best_params

print(f"\nCalibrated Parameters:")
print(f"  ρ_low: {rho_l:.4f}, ρ_high: {rho_h:.4f}")
print(f"  p_regime: {p_r:.4f}, df: {df_opt:.1f}")

Calibrating via grid search (faster)...
  ρ=0.25-0.55, p=0.5, df=4 → err=16.85
  ρ=0.25-0.55, p=0.5, df=6 → err=16.49
  ρ=0.25-0.55, p=0.5, df=8 → err=16.28
  ρ=0.25-0.55, p=0.6, df=4 → err=12.58
  ρ=0.25-0.55, p=0.6, df=6 → err=12.22
  ρ=0.25-0.55, p=0.6, df=8 → err=12.01
  ρ=0.25-0.55, p=0.7, df=4 → err=10.50
  ρ=0.25-0.55, p=0.7, df=6 → err=10.08
  ρ=0.25-0.55, p=0.7, df=8 → err=9.83
  ρ=0.25-0.60, p=0.7, df=6 → err=9.59
  ρ=0.25-0.60, p=0.7, df=8 → err=9.40

Calibrated Parameters:
  ρ_low: 0.2500, ρ_high: 0.6000
  p_regime: 0.7000, df: 8.0


## 6. Final Pricing

In [ ]:
upfront_final, spreads_final = price_all(best_params)

results = [
    ('equity_0_3', f"{upfront_final:.2f} %", f"{target[0]:.2f} %", f"{abs(upfront_final-target[0]):.2f} %"),
    ('mezz_3_7', f"{spreads_final[0]:.2f} bps", f"{target[1]:.2f} bps", f"{abs(spreads_final[0]-target[1]):.2f} bps"),
    ('mezz_7_10', f"{spreads_final[1]:.2f} bps", f"{target[2]:.2f} bps", f"{abs(spreads_final[1]-target[2]):.2f} bps"),
    ('senior_10_15', f"{spreads_final[2]:.2f} bps", f"{target[3]:.2f} bps", f"{abs(spreads_final[2]-target[3]):.2f} bps"),
    ('senior_15_100', f"{spreads_final[3]:.2f} bps", f"{target[4]:.2f} bps", f"{abs(spreads_final[3]-target[4]):.2f} bps")
]

df_result = pd.DataFrame(results, columns=['Tranche', 'Model', 'Market', 'Error'])
print("\nG-VG Copula Pricing Results:")
print(df_result.to_string(index=False))

errors = [abs(upfront_final - target[0])] + [abs(spreads_final[i] - target[i+1]) for i in range(4)]
ape_gvg = sum(errors)
mae_gvg = ape_gvg / 5

print(f"\nAPE: {ape_gvg:.2f}, MAE: {mae_gvg:.2f}")


G-VG Copula Pricing Results:
      Tranche      Model     Market      Error
   equity_0_3    27.98 %    28.30 %     0.33 %
     mezz_3_7 401.41 bps  95.08 bps 306.33 bps
    mezz_7_10 186.99 bps 113.10 bps  73.89 bps
 senior_10_15  99.59 bps  55.94 bps  43.65 bps
senior_15_100   8.34 bps  21.72 bps  13.38 bps

APE: 437.58, MAE: 87.52


## 7. Comparison

In [ ]:
ape_gaussian = gaussian_results['ape']
improvement = (ape_gaussian - ape_gvg) / ape_gaussian * 100

print("\n" + "="*60)
print("MODEL COMPARISON")
print("="*60)
print(f"Gaussian APE: {ape_gaussian:.2f} bps, ρ={gaussian_results['calibrated_correlation']:.4f}")
print(f"G-VG APE: {ape_gvg:.2f} bps, ρ={rho_l:.4f}-{rho_h:.4f}")
print(f"Improvement: {improvement:.1f}% ({ape_gaussian - ape_gvg:.2f} bps reduction)")

base_corr = market_data['base_correlations']
print(f"\nMarket skew: {base_corr['detach_3']:.2f}% → {base_corr['detach_100']:.2f}%")
print(f"G-VG skew: {rho_l*100:.2f}% → {rho_h*100:.2f}% ({(rho_h-rho_l)*100:.2f}pp)")
print("="*60)


MODEL COMPARISON
Gaussian APE: 431.03 bps, ρ=0.3367
G-VG APE: 437.58 bps, ρ=0.2500-0.6000
Improvement: -1.5% (-6.55 bps reduction)

Market skew: 47.67% → 67.56%
G-VG skew: 25.00% → 60.00% (35.00pp)


## 8. Save Results

In [ ]:
with open('../results/results_gvg.json', 'w') as f:
    json.dump({
        'model': 'G-VG Copula',
        'parameters': {'rho_low': rho_l, 'rho_high': rho_h, 'p_regime': p_r, 'df': df_opt},
        'ape': float(ape_gvg),
        'mae': float(mae_gvg),
        'improvement_vs_gaussian': float(improvement)
    }, f, indent=2)

df_result.to_csv('../results/pricing_comparison_gvg.csv', index=False)

pd.DataFrame([
    {'Model': 'Gaussian', 'APE': f"{ape_gaussian:.2f}", 'Skew': '0.00pp'},
    {'Model': 'G-VG', 'APE': f"{ape_gvg:.2f}", 'Skew': f"{(rho_h-rho_l)*100:.2f}pp"},
    {'Model': 'Improvement', 'APE': f"{improvement:.1f}%", 'Skew': '-'}
]).to_csv('../results/model_comparison.csv', index=False)

print("Results saved!")